In [1]:
""""
This script aims to train a digit-recognizer on the mnist handwritten numbers dataset
The tutorial that will be used as a guideline is: https://www.tensorflow.org/tutorials/keras/basic_classification
"""

from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
test_data = pd.read_csv("data/test.csv")
train_data = pd.read_csv("data/train.csv")

y = train_data["label"]
X = train_data.drop(columns=["label"])

# Rescale between 0 and 1
X = X/255.0
# Convert to 28x28 image
test_data = test_data.values.reshape(-1, 28, 28, 1)/255.0

# Convert to numpy arrays and reshape to 28x28 image
X = X.values.reshape(-1, 28, 28, 1)
y = y.values

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [34]:
batch_size = 100

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)
datagen.flow(X_train, y_train, batch_size=batch_size)

In [3]:
# This method comes from a tensorflow CNN tutorial for MNIST. 
# https://www.tensorflow.org/tutorials/estimators/cnn
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  
  # Pooling Layer #2
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=128, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.RMSPropOptimizer(learning_rate=0.0001, epsilon=1e-08, decay=0.0)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  }
  
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

model = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="model/mnist_cnn_model")

INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': 'model/mnist_cnn_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7faa2d212eb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [4]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}


logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [6]:
batch_size = 100

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train,
    batch_size=batch_size,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
model.train(
    input_fn=train_input_fn,
    steps=100)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


INFO:tensorflow:Calling model_fn.


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.


...

INFO:tensorflow:Saving checkpoints for 5490 into model/mnist_cnn_model/model.ckpt.


INFO:tensorflow:loss = 0.000379524048184976, step = 5491


INFO:tensorflow:Saving checkpoints for 5590 into model/mnist_cnn_model/model.ckpt.


Instructions for updating:
Use standard file APIs to delete files with this prefix.


INFO:tensorflow:Loss for final step: 0.00012189070548629388.


In [7]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)

eval_results = model.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2019-08-29T16:59:06Z


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Restoring parameters from model/mnist_cnn_model/model.ckpt-5590


...

INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Finished evaluation at 2019-08-29-16:59:10


INFO:tensorflow:Saving dict for global step 5590: accuracy = 0.9904762, global_step = 5590, loss = 0.049923126


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5590: model/mnist_cnn_model/model.ckpt-5590


{'accuracy': 0.9904762, 'loss': 0.049923126, 'global_step': 5590}


In [20]:
results = model.predict(tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    num_epochs=1,
    shuffle=False))

predictions = []
for result in results:
    predictions.extend(result['classes'].flatten())

predictions = np.asarray(predictions)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Restoring parameters from model/mnist_cnn_model/model.ckpt-5590


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [21]:
image_id = np.arange(1, len(test_data) +1)
d = {"ImageId": image_id, "Label": predictions}
results = pd.DataFrame(data=d, index=image_id).set_index("ImageId")
results.to_csv("data/results.csv")